In [1]:
#This code removes outliers for each PPM

from sklearn.datasets import fetch_openml

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn.linear_model as skl_lm
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score

from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures

from sklearn import utils
from sklearn import preprocessing
from sklearn import neighbors
from pandas import read_csv

from sklearn.feature_selection import SelectKBest, f_classif

from sklearn import preprocessing

from sklearn.feature_selection import SelectKBest

import csv
lda = LinearDiscriminantAnalysis()
qda = QuadraticDiscriminantAnalysis()
kfld = KFold(n_splits=3,random_state=42)

In [2]:
#Reads data in from data folder.


data = read_csv('data/ethylene_methane.txt',delim_whitespace=True)
print(data.shape)

(4178504, 19)


In [3]:
#creates a new attribute thats the % Methane, in cases where we divide by zero
#we recieve a NAN which is replaced by a 0
#this code also finds all unique PPM for methane

data['Methane%'] = (data['Methane(ppm)']/(data['Methane(ppm)']+data['Ethylene(ppm)']))
data['Methane%'] = data['Methane%'].fillna(0)

ppm = data['Methane(ppm)'].unique()
sensor_name = ['sensor1', 'sensor2', 'sensor3', 'sensor4', 'sensor5', 'sensor6', 'sensor7', 'sensor8', 'sensor9', 'sensor10', 'sensor11', 'sensor12', 'sensor13', 'sensor14', 'sensor15', 'sensor16']

In [4]:
#This block searches for Outliers for each PPM of Methane and then drops them
#from the file. Skip this line if you want to produce a result with outliers

for j in sensor_name:
    print(j)
    for i in np.sort(ppm):    
        data2 = data[data["Methane(ppm)"] == i] #only select a specific PPM to begin trimming
        
        results = data2[j].quantile([0.125,0.875]) #Next lines calculate the range for non-Outliers
        IQR_15 = 1.5*(results.loc[0.875] - results.loc[0.125])
        fq = results.loc[0.125] - IQR_15
        tq = results.loc[0.875] + IQR_15
                
        index = data2[(fq>data2[j]) | (tq<data2[j])] #Finds Outliers
        data = data.drop(index.index) #Uses index of Outliers to drop rows from data 
        
        
        #print('Methane(ppm) = ',i)
        #print('Outlier range',fq , tq)
        #print(data.shape)
        #print("")

sensor1
sensor2
sensor3
sensor4
sensor5
sensor6
sensor7
sensor8
sensor9
sensor10
sensor11
sensor12
sensor13
sensor14
sensor15
sensor16


In [5]:
#Defines our X and y and sets y equal to 1 if the Methane % is greater than 50%
#splits the data into train and test then runs it for Logistic Regression to 
#get a reference R2 value


X = preprocessing.scale(data.drop(columns = ['Methane(ppm)','Ethylene(ppm)','Methane%']))
y = np.where(data['Methane%'] >= 0.5, 1, 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42) 

clf = LogisticRegression(random_state=42, solver='lbfgs',max_iter=10000,n_jobs = -1).fit(X_train, y_train)
cross_val_score(clf, X_test, y_test, cv=kfld, scoring='r2').mean()

#R2 b4 cleaning = 0.7601212781453967
#R2 after = 0.919442734823846

0.919442734823846